## ANALISIS DE DATOS DE ACC (ACELEROMETRO)

El presente es para analizar los datos del acelerometro del smartwatch, el cual tiene un procesamiento de datos en 32Hz, que seria la 32ava parte de un segundo

In [16]:
# Importando Pandas y otras librerias
import pandas as pd
import numpy as np

In [17]:
# Leyendo el CSV
acelerometro_values = pd.read_csv('D://Jesus//UNSA//Maestria//Tesis//Codigo Test//PreTesisMasterRepo//ACC//ACC_016.csv', engine='python', na_values="not available")

In [18]:
acelerometro_values.head()

,datetime,acc_x,acc_y,acc_z
0,2020-07-16 09:29:03.000000,-39.0,-28.0,37.0
1,2020-07-16 09:29:03.031250,-38.0,-27.0,37.0
2,2020-07-16 09:29:03.062500,-37.0,-37.0,37.0
3,2020-07-16 09:29:03.093750,-47.0,-27.0,41.0
4,2020-07-16 09:29:03.125000,-39.0,-37.0,34.0


In [19]:
acelerometro_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17860788 entries, 0 to 17860787
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   datetime  object 
 1    acc_x    float64
 2    acc_y    float64
 3    acc_z    float64
dtypes: float64(3), object(1)
memory usage: 545.1+ MB


In [20]:
acelerometro_values.count()

datetime    17860788
 acc_x      17860788
 acc_y      17860788
 acc_z      17860788
dtype: int64

In [21]:
acelerometro_values["datetime"].head()

0    2020-07-16 09:29:03.000000
1    2020-07-16 09:29:03.031250
2    2020-07-16 09:29:03.062500
3    2020-07-16 09:29:03.093750
4    2020-07-16 09:29:03.125000
Name: datetime, dtype: object

### Trabajando con Datetime
Lo primero sera convertir los datetime a el formato correcto, ya que lo esta detectando como object, lo siguiente sera colocar como index las fechas y al final agrupar por cada 5 minutos los datos para obtener el promedio y media de los datos


In [22]:
# Convertimos en fechas los datimetimes
acelerometro_values['datetime'] = pd.to_datetime(acelerometro_values['datetime'])

In [28]:
print(acelerometro_values.columns)

## Añadimos el valor de la magnitud
acelerometro_values["magnitude"] = np.sqrt(acelerometro_values[' acc_x']**2 + acelerometro_values[' acc_y']**2 + acelerometro_values[' acc_z']**2)
acelerometro_values = acelerometro_values.set_index('datetime')
acelerometro_values.head()


Index(['datetime', ' acc_x', ' acc_y', ' acc_z'], dtype='object')


,acc_x,acc_y,acc_z,magnitude
datetime,,,,
2020-07-16 09:29:03.000000,-39.0,-28.0,37.0,60.613530
2020-07-16 09:29:03.031250,-38.0,-27.0,37.0,59.514704
2020-07-16 09:29:03.062500,-37.0,-37.0,37.0,64.085880
2020-07-16 09:29:03.093750,-47.0,-27.0,41.0,67.963225
2020-07-16 09:29:03.125000,-39.0,-37.0,34.0,63.608176


In [38]:
df_procesado_5min = acelerometro_values.resample('5min') 

### Calculamos la magnitud del vector

En este caso tenemos que encontrar la magnitud del vector, en este caso en 3D, donde obtendremos el promedio

In [39]:
# Obtenemos el promedio
df_promedio_5min = df_procesado_5min.agg(['mean', 'std'])

# Removemos las columnas que no necesitamos por ahora
# Supongamos que tienes tus datos en un DataFrame llamado 'df'
columns_to_remove = [' acc_x', ' acc_y', ' acc_z']
df_promedio_5min = df_promedio_5min.drop(columns=columns_to_remove)
df_promedio_5min.head(20)

magnitude           
                          mean        std
datetime                                 
2020-07-16 09:25:00  65.035743  11.721119
2020-07-16 09:30:00  62.730299   3.857899
2020-07-16 09:35:00  62.568438   3.254793
2020-07-16 09:40:00  62.401744   3.242671
2020-07-16 09:45:00  62.548595   2.355077
2020-07-16 09:50:00  63.158727   3.646927
2020-07-16 09:55:00  63.193804   4.738275
2020-07-16 10:00:00  64.353551  10.946620
2020-07-16 10:05:00  64.282381  14.525395
2020-07-16 10:10:00  64.558886  13.317263
2020-07-16 10:15:00  63.573932   4.787022
2020-07-16 10:20:00  63.311439   5.513508
2020-07-16 10:25:00  63.549230   5.018105
2020-07-16 10:30:00  63.766912   2.047082
2020-07-16 10:35:00  63.778592   2.923121
2020-07-16 10:40:00  62.982472   2.173442
2020-07-16 10:45:00  62.857097   3.302068
2020-07-16 10:50:00  64.245831   9.318329
2020-07-16 10:55:00  63.132467   6.579583
2020-07-16 11:00:00  62.987030   4.358714

In [37]:
# Lo mismo aplicamos para 1 hora
df_procesado_1hora = acelerometro_values.resample('1h') 
# Obtenemos el promedio
df_promedio_1hora = df_procesado_1hora.agg(['mean', 'std'])

# Removemos las columnas que no necesitamos por ahora
# Supongamos que tienes tus datos en un DataFrame llamado 'df'
columns_to_remove = [' acc_x', ' acc_y', ' acc_z']
df_promedio_1hora = df_promedio_1hora.drop(columns=columns_to_remove)
df_promedio_1hora.head(20)

magnitude          
                          mean       std
datetime                                
2020-07-16 09:00:00  62.836575  4.116197
2020-07-16 10:00:00  63.699399  7.898439
2020-07-16 11:00:00  63.038655  4.180281
2020-07-16 12:00:00  63.215403  5.198428
2020-07-16 13:00:00  63.371440  6.171390
2020-07-16 14:00:00  63.009526  4.205482
2020-07-16 15:00:00  62.996885  5.131926
2020-07-16 16:00:00  63.028031  5.624842
2020-07-16 17:00:00  63.658786  6.270833
2020-07-16 18:00:00  63.284474  3.744750
2020-07-16 19:00:00  63.313038  5.859518
2020-07-16 20:00:00  63.448638  6.388613
2020-07-16 21:00:00  63.356545  4.593087
2020-07-16 22:00:00  63.209259  5.082544
2020-07-16 23:00:00  63.353230  5.023345
2020-07-17 00:00:00  63.777779  0.669736
2020-07-17 01:00:00  63.441064  1.015482
2020-07-17 02:00:00  62.613963  1.392232
2020-07-17 03:00:00  63.609066  3.061001
2020-07-17 04:00:00  63.410153  1.523511

In [40]:
# Exportamos los resultados en un csv
df_promedio_5min.to_csv("ACC_5min.csv")
df_promedio_1hora.to_csv("ACC_1hora.csv")